In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import calendar
import warnings
from math import sin, cos, sqrt, atan2, radians,asin

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb

warnings.filterwarnings('ignore')

In [119]:
train=pd.read_csv("train.csv",nrows=3000000)
print("Shape of Training Data",train.shape)
test=pd.read_csv("test.csv")
print("Shape of Testing Data", test.shape)



Shape of Training Data (3000000, 8)
Shape of Testing Data (9914, 7)


### Clean the data 
1. Remove fare amount < 0 
2. Passenger count > 7 remove from train data
3. Use only the fields that are present to create Baseline Model - have only date features. Rest of the features we will add later

In [120]:
def encodeDays(day_of_week):
    day_dict={'Sunday':0,'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6}
    return day_dict[day_of_week]
def clean_data(data):
    boundary={'min_lng':-74.263242,
              'min_lat':40.573143,
              'max_lng':-72.986532, 
              'max_lat':41.709555}
    
    data['pickup_datetime']=pd.to_datetime(data['pickup_datetime'],format='%Y-%m-%d %H:%M:%S UTC')
    data['pickup_day']=data['pickup_datetime'].apply(lambda x:x.day)
    data['pickup_hour']=data['pickup_datetime'].apply(lambda x:x.hour)
    data['pickup_day_of_week']=data['pickup_datetime'].apply(lambda x:calendar.day_name[x.weekday()])
    data['pickup_month']=data['pickup_datetime'].apply(lambda x:x.month)
    data['pickup_year']=data['pickup_datetime'].apply(lambda x:x.year)
    if 'fare_amount' in data.columns:
        data=data[data['fare_amount']>=0]
        data.loc[~((data.pickup_longitude >= boundary['min_lng'] ) & (data.pickup_longitude <= boundary['max_lng']) &
            (data.pickup_latitude >= boundary['min_lat']) & (data.pickup_latitude <= boundary['max_lat']) &
            (data.dropoff_longitude >= boundary['min_lng']) & (data.dropoff_longitude <= boundary['max_lng']) &
            (data.dropoff_latitude >=boundary['min_lat']) & (data.dropoff_latitude <= boundary['max_lat'])),'is_outlier_loc']=1
        data.loc[((data.pickup_longitude >= boundary['min_lng'] ) & (data.pickup_longitude <= boundary['max_lng']) &
            (data.pickup_latitude >= boundary['min_lat']) & (data.pickup_latitude <= boundary['max_lat']) &
            (data.dropoff_longitude >= boundary['min_lng']) & (data.dropoff_longitude <= boundary['max_lng']) &
            (data.dropoff_latitude >=boundary['min_lat']) & (data.dropoff_latitude <= boundary['max_lat'])),'is_outlier_loc']=0

    #print("Outlier vs Non Outlier Counts")
    #print(data['is_outlier_loc'].value_counts())

    # Let us drop rows, where location is outlier
        data=data.loc[data['is_outlier_loc']==0]
        data.drop(['is_outlier_loc'],axis=1,inplace=True)
    
    data=data[data['passenger_count']<=8]
    data['pickup_day_of_week']=data['pickup_day_of_week'].apply(lambda x:encodeDays(x))
    return data

In [121]:
train=clean_data(train)
test=clean_data(test)
print("Shape of Training Data after cleaning ",train.shape)
print("Shape of Testing Data after cleaning", test.shape)

Shape of Training Data after cleaning  (2935680, 13)
Shape of Testing Data after cleaning (9914, 12)


### Create a function to process data for Modelling
This step includes:
1. Dropping unwanted columns from the data
2. One Hot Encoding of categorical variables
3. Dividing training data into train and validation data sets
    1. features and target varible must be seperated
    2. split ratio must be passed as an argument 

In [122]:
def processDataForModelling(data,target,drop_cols,is_train=True,split=0.25):
    data_1=data.drop(drop_cols,axis=1)
    # One hot Encoding
    data_1=pd.get_dummies(data_1)
    if is_train==True:
        X=data_1.drop([target],axis=1)
        y=data_1[target]
        X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=split,random_state=123)
        
        print("Shape of Training Features",X_train.shape)
        print("Shape of Validation Features ",X_test.shape)
        
        return X_train, X_test, y_train, y_test
    else:
        print ("Shape of Test Data",data_1.shape)
        return data_1
    

In [123]:
X_train, X_test, y_train, y_test=processDataForModelling(train,'fare_amount',drop_cols=['key','pickup_datetime'],is_train=True,split=0.2)

Shape of Training Features (2348544, 10)
Shape of Validation Features  (587136, 10)


In [124]:
test_data=processDataForModelling(test,'fare_amount',drop_cols=['key','pickup_datetime'],is_train=False)

Shape of Test Data (9914, 10)


### Building a Baseline Model and Identifying a good ML algorithm for this problem
The metric used in this problem is RMSE. 
We will try three models - Linear Regression, Random Forest and XGBoost and see which model performs better.
We will use the best model among the three to further tune and apply feature Engineering

For Baseline, we will predict the average fare amount and check the RMSE on validation data. Any model, should be able to beat this simple benchmark

In [8]:
avg_fare=round(np.mean(y_train),2)
avg_fare

11.31

In [9]:
baseline_pred=np.repeat(avg_fare,y_test.shape[0])
baseline_rmse=np.sqrt(mean_squared_error(baseline_pred, y_test))
print("Basline RMSE of Validation data :",baseline_rmse)

Basline RMSE of Validation data : 9.707556295335856


##### Build a Linear Regression Model 

In [10]:
lm = LinearRegression()
lm.fit(X_train,y_train)
y_pred=np.round(lm.predict(X_test),2)
lm_rmse=np.sqrt(mean_squared_error(y_pred, y_test))
print("RMSE for Linear Regression is ",lm_rmse)

RMSE for Linear Regression is  8.239673655728213


The linear regression model performed better than than the Baseline Mode. Let us create a Random Forest Model and see how it performs. Reason for failure of logistic regression model, is that it tries to fit a linear line between the variables and the targer. But, as we saw in the Exploratory analysis phase this is not true. 

##### Build a Random Forest Model 

In [131]:
rf = RandomForestRegressor(n_estimators = 100, random_state = 883,n_jobs=-1)
rf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=883, verbose=0, warm_start=False)

In [132]:
rf_pred= rf.predict(X_test)
rf_rmse=np.sqrt(mean_squared_error(rf_pred, y_test))
print("RMSE for Random Forest is ",rf_rmse)

RMSE for Random Forest is  3.722760643979709


Random Forest has reduced the RMSE considerably as compared to Linear Regression. Random Forest works on the principle of Bagging
The idea behind this is very intutive- when we want to make a decision about a field we do not know about we take advice from a people and then we decide based on the majority opinion.
This is the idea behind Random Forest - multiple decision trees are created and output from each of the trees are averaged to predict the value.
Random Forest are not susceptible to overfitting and since in Random Forest, each tree is trained independently of the other, it is more robust.

The next type of algorithm we will see is also another Tree based Ensemble algorithm - but it follows concept of Boosting. 




#### Building LightGBM Algorithm

In [125]:
train_data=lgb.Dataset(X_train,label=y_train)


In [133]:
param = {'num_leaves':31, 'num_trees':5000, 'objective':'regression'}
param['metric'] = 'l2_root'

In [134]:
num_round=5000
cv_results = lgb.cv(param, train_data, num_boost_round=num_round, nfold=5,verbose_eval=20, early_stopping_rounds=20,stratified=False)

[20]	cv_agg's rmse: 5.06279 + 0.05548
[40]	cv_agg's rmse: 4.50898 + 0.0472365
[60]	cv_agg's rmse: 4.30951 + 0.0631628
[80]	cv_agg's rmse: 4.21842 + 0.0587082
[100]	cv_agg's rmse: 4.15903 + 0.0561534
[120]	cv_agg's rmse: 4.11538 + 0.0551337
[140]	cv_agg's rmse: 4.08347 + 0.0569919
[160]	cv_agg's rmse: 4.05952 + 0.0571602
[180]	cv_agg's rmse: 4.0394 + 0.0586051
[200]	cv_agg's rmse: 4.02268 + 0.0549178
[220]	cv_agg's rmse: 4.00745 + 0.0558276
[240]	cv_agg's rmse: 3.99439 + 0.0557403
[260]	cv_agg's rmse: 3.98173 + 0.0563307
[280]	cv_agg's rmse: 3.96865 + 0.0567871
[300]	cv_agg's rmse: 3.95948 + 0.0574369
[320]	cv_agg's rmse: 3.95052 + 0.0568952
[340]	cv_agg's rmse: 3.94325 + 0.0560929
[360]	cv_agg's rmse: 3.93594 + 0.056963
[380]	cv_agg's rmse: 3.9317 + 0.0577162
[400]	cv_agg's rmse: 3.92711 + 0.0572581
[420]	cv_agg's rmse: 3.921 + 0.0572053
[440]	cv_agg's rmse: 3.91641 + 0.0573622
[460]	cv_agg's rmse: 3.91227 + 0.0583486
[480]	cv_agg's rmse: 3.90848 + 0.0594168
[500]	cv_agg's rmse: 3.9046

In [135]:
print('Best num_boost_round:', len(cv_results['rmse-mean']))
#lgb_pred = lgb_bst.predict(X_test)

Best num_boost_round: 1560


In [136]:
lgb_bst=lgb.train(param,train_data,len(cv_results['rmse-mean']))

In [137]:
lgb_pred = lgb_bst.predict(X_test)
lgb_rmse=np.sqrt(mean_squared_error(lgb_pred, y_test))
print("RMSE for Light GBM is ",lgb_rmse)

RMSE for Light GBM is  3.8028190168518723


LighGBM was much quicker than RF and the RMSE is quiet close to RF too. Tuning this can get better results. For the rest of this excercise we will go with LIGHT GBM

#### Feature Engineering

In [138]:
nyc_airports={'JFK':{'min_lng':-73.8352,
     'min_lat':40.6195,
     'max_lng':-73.7401, 
     'max_lat':40.6659},
              
    'EWR':{'min_lng':-74.1925,
            'min_lat':40.6700, 
            'max_lng':-74.1531, 
            'max_lat':40.7081

        },
    'LaGuardia':{'min_lng':-73.8895, 
                  'min_lat':40.7664, 
                  'max_lng':-73.8550, 
                  'max_lat':40.7931
        
    }
    
}
def isAirport(latitude,longitude,airport_name='JFK'):
    
    if latitude>=nyc_airports[airport_name]['min_lat'] and latitude<=nyc_airports[airport_name]['max_lat'] and longitude>=nyc_airports[airport_name]['min_lng'] and longitude<=nyc_airports[airport_name]['max_lng']:
        return 1
    else:
        return 0

In [139]:
nyc_boroughs={
    'manhattan':{
        'min_lng':-74.0479,
        'min_lat':40.6829,
        'max_lng':-73.9067,
        'max_lat':40.8820
    },
    
    'queens':{
        'min_lng':-73.9630,
        'min_lat':40.5431,
        'max_lng':-73.7004,
        'max_lat':40.8007

    },

    'brooklyn':{
        'min_lng':-74.0421,
        'min_lat':40.5707,
        'max_lng':-73.8334,
        'max_lat':40.7395

    },

    'bronx':{
        'min_lng':-73.9339,
        'min_lat':40.7855,
        'max_lng':-73.7654,
        'max_lat':40.9176

    },

    'staten_island':{
        'min_lng':-74.2558,
        'min_lat':40.4960,
        'max_lng':-74.0522,
        'max_lat':40.6490

    }
}

In [140]:
def getBorough(lat,lng):
    
    locs=nyc_boroughs.keys()
    for loc in locs:
        if lat>=nyc_boroughs[loc]['min_lat'] and lat<=nyc_boroughs[loc]['max_lat'] and lng>=nyc_boroughs[loc]['min_lng'] and lng<=nyc_boroughs[loc]['max_lng']:
            return loc
    return 'others'

In [141]:
lower_manhattan_boundary={'min_lng': -74.0194,
                          'min_lat':40.6997,
                          'max_lng':-73.9716,
                          'max_lat':40.7427}

def isLowerManhattan(lat,lng):
    if lat>=lower_manhattan_boundary['min_lat'] and lat<=lower_manhattan_boundary['max_lat'] and lng>=lower_manhattan_boundary['min_lng'] and lng<=lower_manhattan_boundary['max_lng']:
        return 1
    else:
        return 0

In [142]:
train['is_pickup_lower_manhattan']=train.apply(lambda row:isLowerManhattan(row['pickup_latitude'],row['pickup_longitude']),axis=1)
train['is_dropoff_lower_manhattan']=train.apply(lambda row:isLowerManhattan(row['dropoff_latitude'],row['dropoff_longitude']),axis=1)

In [143]:
test['is_pickup_lower_manhattan']=test.apply(lambda row:isLowerManhattan(row['pickup_latitude'],row['pickup_longitude']),axis=1)
test['is_dropoff_lower_manhattan']=test.apply(lambda row:isLowerManhattan(row['dropoff_latitude'],row['dropoff_longitude']),axis=1)


In [144]:
def distance(lat1,lon1,lat2,lon2):
    p = 0.017453292519943295 # Pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 0.6213712 * 12742 * np.arcsin(np.sqrt(a))
train['trip_distance']=train.apply(lambda row:distance(row['pickup_latitude'],row['dropoff_latitude'],row['pickup_longitude'],row['dropoff_longitude']),axis=1)
test['trip_distance']=test.apply(lambda row:distance(row['pickup_latitude'],row['dropoff_latitude'],row['pickup_longitude'],row['dropoff_longitude']),axis=1)

lgr=(-73.8733, 40.7746)
jfk=(-73.7900, 40.6437)
ewr=(-74.1843, 40.6924)

test['pickup_distance_jfk']=test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],jfk[1],jfk[0]),axis=1)
test['dropoff_distance_jfk']=test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],jfk[1],jfk[0]),axis=1)
test['pickup_distance_ewr']=test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],ewr[1],ewr[0]),axis=1)
test['dropoff_distance_ewr']=test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],ewr[1],ewr[0]),axis=1)
test['pickup_distance_laguardia']=test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],lgr[1],lgr[0]),axis=1)
test['dropoff_distance_laguardia']=test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],lgr[1],lgr[0]),axis=1)



train['pickup_distance_jfk']=train.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],jfk[1],jfk[0]),axis=1)
train['dropoff_distance_jfk']=train.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],jfk[1],jfk[0]),axis=1)
train['pickup_distance_ewr']=train.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],ewr[1],ewr[0]),axis=1)
train['dropoff_distance_ewr']=train.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],ewr[1],ewr[0]),axis=1)
train['pickup_distance_laguardia']=train.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],lgr[1],lgr[0]),axis=1)
train['dropoff_distance_laguardia']=train.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],lgr[1],lgr[0]),axis=1)

In [145]:
manhattan=(-73.9664, 40.7909)
queens=(-73.8317, 40.7038)
brooklyn=(-73.9489, 40.6551)
bronx=(-73.8568, 40.8572)
staten_island=(-74.1540, 40.5725)




test['pickup_distance_manhattan']=test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],manhattan[1],manhattan[0]),axis=1)
test['pickup_distance_queens']=test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],queens[1],queens[0]),axis=1)
test['pickup_distance_brooklyn']=test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],brooklyn[1],brooklyn[0]),axis=1)
test['pickup_distance_bronx']=test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],bronx[1],bronx[0]),axis=1)
test['pickup_distance_statenisland']=test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],staten_island[1],staten_island[0]),axis=1)





test['dropoff_distance_manhattan']=test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],manhattan[1],manhattan[0]),axis=1)
test['dropoff_distance_queens']=test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],queens[1],queens[0]),axis=1)
test['dropoff_distance_brooklyn']=test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],brooklyn[1],brooklyn[0]),axis=1)
test['dropoff_distance_bronx']=test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],bronx[1],bronx[0]),axis=1)
test['dropoff_distance_statenisland']=test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],staten_island[1],staten_island[0]),axis=1)

train['pickup_distance_manhattan']=train.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],manhattan[1],manhattan[0]),axis=1)
train['pickup_distance_queens']=train.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],queens[1],queens[0]),axis=1)
train['pickup_distance_brooklyn']=train.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],brooklyn[1],brooklyn[0]),axis=1)
train['pickup_distance_bronx']=train.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],bronx[1],bronx[0]),axis=1)
train['pickup_distance_statenisland']=train.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],staten_island[1],staten_island[0]),axis=1)

train['dropoff_distance_manhattan']=train.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],manhattan[1],manhattan[0]),axis=1)
train['dropoff_distance_queens']=train.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],queens[1],queens[0]),axis=1)
train['dropoff_distance_brooklyn']=train.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],brooklyn[1],brooklyn[0]),axis=1)
train['dropoff_distance_bronx']=train.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],bronx[1],bronx[0]),axis=1)
train['dropoff_distance_statenisland']=train.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],staten_island[1],staten_island[0]),axis=1)

train.to_csv("train_cleaned.csv")
test.to_csv("test_cleaned.csv")

In [146]:
###  We will apply same steps as above

X_train, X_test, y_train, y_test=processDataForModelling(train,'fare_amount',drop_cols=['key','pickup_datetime'],is_train=True,split=0.2)

Shape of Training Features (2348544, 29)
Shape of Validation Features  (587136, 29)


In [147]:
test_data=processDataForModelling(test,'fare_amount',drop_cols=['key','pickup_datetime'],is_train=False)

Shape of Test Data (9914, 29)


In [148]:
train_data=lgb.Dataset(X_train,label=y_train)
param = {'num_leaves':31, 'num_trees':1000, 'objective':'regression'}
param['metric'] = 'l2_root'

In [149]:
num_round=5000
cv_results = lgb.cv(param, train_data, num_boost_round=num_round, nfold=5,verbose_eval=20, early_stopping_rounds=20,stratified=False)

[20]	cv_agg's rmse: 4.93857 + 0.0361967
[40]	cv_agg's rmse: 4.35581 + 0.0493811
[60]	cv_agg's rmse: 4.15574 + 0.0491483
[80]	cv_agg's rmse: 4.04744 + 0.0465886
[100]	cv_agg's rmse: 3.98069 + 0.0439523
[120]	cv_agg's rmse: 3.93519 + 0.0456173
[140]	cv_agg's rmse: 3.90411 + 0.0445432
[160]	cv_agg's rmse: 3.87923 + 0.0431755
[180]	cv_agg's rmse: 3.86033 + 0.0439698
[200]	cv_agg's rmse: 3.84439 + 0.0474349
[220]	cv_agg's rmse: 3.83107 + 0.0493404
[240]	cv_agg's rmse: 3.81917 + 0.0485925
[260]	cv_agg's rmse: 3.80654 + 0.048883
[280]	cv_agg's rmse: 3.79663 + 0.0477744
[300]	cv_agg's rmse: 3.78913 + 0.0483084
[320]	cv_agg's rmse: 3.78244 + 0.0486047
[340]	cv_agg's rmse: 3.7774 + 0.0484563
[360]	cv_agg's rmse: 3.77096 + 0.048162
[380]	cv_agg's rmse: 3.7657 + 0.0479438
[400]	cv_agg's rmse: 3.7622 + 0.0481963
[420]	cv_agg's rmse: 3.75704 + 0.0471727
[440]	cv_agg's rmse: 3.75243 + 0.0469251
[460]	cv_agg's rmse: 3.74771 + 0.0465064
[480]	cv_agg's rmse: 3.74503 + 0.0465838
[500]	cv_agg's rmse: 3.74

In [150]:
print('Best num_boost_round:', len(cv_results['rmse-mean']))

Best num_boost_round: 1000


In [155]:
lgb_bst=lgb.train(param,train_data,len(cv_results['rmse-mean']))

In [156]:
lgb_pred = lgb_bst.predict(X_test)
lgb_rmse=np.sqrt(mean_squared_error(lgb_pred, y_test))
print("RMSE for Light GBM with Feature Engineering is ",lgb_rmse)

RMSE for Light GBM with Feature Engineering is  3.6642689727498543


This is better than without features , we can tune this model. The RMSE has decreased from 3.8 to 3.6. Let us get the best model by tuning the parameters. But, before that let us check what are the important features in our model and whether some features are not helping our model. But we saw above that Random Forest does better than Light GBM.So let us use rf in Light GBM

In [172]:
lgb_params = {'num_leaves':31,'num_trees':5000,
'objective':'regression',
"boosting":"rf",
"bagging_freq":1,
 "bagging_fraction":0.4}

In [173]:
cv_results = lgb.cv(lgb_params, train_data, num_boost_round=num_round, nfold=5,verbose_eval=20, early_stopping_rounds=20,stratified=False)

LightGBMError: Check failed: config->feature_fraction < 1.0f && config->feature_fraction > 0.0f at c:\bld\lightgbm_1531588417232\work\compile\src\boosting\rf.hpp, line 30 .
